In [ ]:
pip install -U lightautoml

     |████████████████████████████████| 262 kB 8.8 MB/s 
     |████████████████████████████████| 302 kB 43.6 MB/s 
     |████████████████████████████████| 232 kB 56.3 MB/s 
     |████████████████████████████████| 2.6 MB 47.4 MB/s 
     |████████████████████████████████| 24.0 MB 90 kB/s 
     |████████████████████████████████| 67.4 MB 28 kB/s 
     |████████████████████████████████| 98 kB 10.0 MB/s 
     |████████████████████████████████| 204 kB 61.9 MB/s 
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 1.2 MB 68.0 MB/s 
     |████████████████████████████████| 420 kB 64.2 MB/s 
     |████████████████████████████████| 37.1 MB 49 kB/s 
     |████████████████████████████████| 9.1 MB 51.9 MB/s 
     |████████████████████████████████| 636 kB 62.8 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 164 kB 62.6 MB/s 
     |████████████

In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=be413db72be23045473721b3788f1baecd547f7eaedd78f245dd8a919ac53fde
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget
import pandas as pd
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
import requests
import json

In [ ]:
states = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

states.keys()

dict_keys(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO'])

Os dados de CNPJ considerados potencialmente poluidores estão divididos em UFs, devido ao tamanho dos datasets. Fez-se a agregação deles por um script de download e merge.

In [ ]:
for i in tqdm(states.keys()):
  url = f'http://dadosabertos.ibama.gov.br/dados/CTF/APP/{i}/pessoasJuridicas.csv'
  wget.download(url, f'/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/cnpjs_potenciais_{i}.csv')

cols = ['CNPJ', 'Razão Social', 'Código da categoria', 'Descrição da categoria',
       'Código da atividade', 'Descrição da atividade',
       'Data de início da atividade', 'Data de término da atividade',
       'Potenc. de Pol. da atividade', 'Município', 'Estado', 'Latitude',
       'Longitude', 'Situação cadastral', 'Última Atualização Relatório']

poten_cnpj = pd.DataFrame(columns = cols)

for i in tqdm(states.keys()):
  poten_cnpj = poten_cnpj.append(pd.read_csv(f"/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/cnpjs_potenciais_{i}.csv", sep=";"))

poten_cnpj.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/cnpjs_potencias_agg.csv")


In [ ]:
### download dados de operacoes automaticas

url = 'https://dadosabertos.bndes.gov.br/dataset/0f335c85-92a8-4343-9423-f073fb40774e/resource/9534f677-9525-4bf8-a3aa-fd5d3e152a93/download/operacoes-financiamento-operacoes-indiretas-automaticas.csv'
wget.download(url, '/content/drive/MyDrive/BNDES_premio_dados/dados_operacoes/oper_automaticas.csv')

### Import dados

Dados de CNPJs potencialmente poluidores, agregados.

In [ ]:
poten_cnpj = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/cnpjs_potencias_agg.csv")
poten_cnpj["potenc_poluic"] = "s"
poten_cnpj = poten_cnpj.rename(columns = {"CNPJ": "cnpj", "Razão Social": "razao_social"})

poten_cnpj.head()

,Unnamed: 0,cnpj,razao_social,Código da categoria,Descrição da categoria,Código da atividade,Descrição da atividade,Data de início da atividade,Data de término da atividade,Potenc. de Pol. da atividade,Município,Estado,Latitude,Longitude,Situação cadastral,Última Atualização Relatório,potenc_poluic
0,0,240681000165,FRIOS VILHENA IMPORTAÇÃO E EXPORTAÇÃO LTDA,18,"Transporte, Terminais, Depósitos e Comércio",11,Transporte de produtos florestais,23/04/2014,29/06/2018,...,RIO BRANCO,ACRE,"-9,981275","-67,809894",Ativa,18/08/2021 19:00,s
1,1,240681000165,FRIOS VILHENA IMPORTAÇÃO E EXPORTAÇÃO LTDA,18,"Transporte, Terminais, Depósitos e Comércio",28,depósito de mercadorias para terceiros exceto ...,08/11/2012,23/04/2014,...,RIO BRANCO,ACRE,"-9,981275","-67,809894",Ativa,18/08/2021 19:00,s
2,2,240681000165,FRIOS VILHENA IMPORTAÇÃO E EXPORTAÇÃO LTDA,21,Atividades não relacionadas no Anexo VIII da L...,1,reparação de aparelhos de refrigeração,06/12/2011,15/02/2013,...,RIO BRANCO,ACRE,"-9,981275","-67,809894",Ativa,18/08/2021 19:00,s
3,3,240681000165,FRIOS VILHENA IMPORTAÇÃO E EXPORTAÇÃO LTDA,21,Atividades não relacionadas no Anexo VIII da L...,49,Transporte de produtos florestais - Lei nº 12....,29/06/2018,NaN,...,RIO BRANCO,ACRE,"-9,981275","-67,809894",Ativa,18/08/2021 19:00,s
4,4,240681000246,FRIOS VILHENA IMP. E EXP. LTDA,18,"Transporte, Terminais, Depósitos e Comércio",11,Transporte de produtos florestais,23/04/2014,29/06/2018,...,BRASILEIA,ACRE,"-11,006917","-68,7425",Ativa,18/08/2021 19:00,s


Dados de operações automáticas.

In [ ]:
oper_automaticas = pd.read_csv('/content/drive/MyDrive/BNDES_premio_dados/dados_operacoes/oper_automaticas.csv', sep=";")
oper_automaticas = oper_automaticas.rename(columns = {"cpf_cnpj": "cnpj_bndes"})

oper_automaticas.head()

,cliente,cnpj_bndes,uf,municipio,municipio_codigo,data_da_contratacao,valor_da_operacao_em_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,forma_de_apoio,produto,instrumento_financeiro,inovacao,area_operacional,setor_cnae,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_do_agente_financeiro,situacao_da_operacao
0,BANCO COOPERATIVO SICOOB S.A.,**.*38.232/0001-**,SP,PEDREGULHO,3537008,2002-01-02,16000,"16000,0",RECURSOS LIVRES - TESOURO,TAXA FIXA,"8,75",24,36,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,OUTROS,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO COOPERATIVO SICOOB S.A.,02.038.232.0001-64,LIQUIDADA
1,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,TRES DE MAIO,4321808,2002-01-02,8047,"8047,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
2,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,8104,"8104,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
3,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,6304,"6304,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA
4,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,HUMAITA,4309704,2002-01-02,15000,"15000,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA


O dataset abaixo se trata da técnica utilizada para extrair os cnpjs ocultados da base de operações indiretas. Utilizou-se a técnica de [distancia de levenshtein](https://medium.com/@everton.tomalok/calculando-similaridades-entre-strings-ebbea21d5b7a) para calcular similaridade de strings, e assim consegui identificar qual CNPJ da base da Receita Federal é mais parecido com aqueles ocultados na base do BNDES.

Todo esse processo foi feito via Big Query, com ajuda de dois projetos open source: [BaseDosDados](https://basedosdados.org/about) para utilizar o big query e [brasil.io](https://brasil.io/home/) que compilou os dados de CNPJs.

In [ ]:
cnpj_receita = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/notebooks/luis_depra_cnpj_bndes_cnpj.csv")
cnpj_receita["cnpj_receita"] = cnpj_receita[cnpj_receita["cnpj_receita"].notnull()]["cnpj_receita"].astype(int).astype(str)
cnpj_receita = cnpj_receita.rename(columns = {"nome_bndes": "cliente"})

cnpj_receita.head()

,Unnamed: 0,cliente,nome_receita,cnpj_bndes,cnpj_receita,score,rank
0,0,A B FILHO TRANSPORTES,TRANSLOG. TRANSPORTES E CARGAS LTDA,**.*22.234/0001-**,11522234000115,69.0,1
1,1,A.G.L. AUTOMACAO LTDA - ME,G R M COMERCIO LTDA,**.*25.948/0001-**,35525948000145,62.0,1
2,2,ANTONINHO SUPERMERCADO LTDA,ANTONINHO ATACADO E VAREJO LTDA,**.*94.231/0001-**,894231000196,68.0,1
3,3,BORRACHAS PLANALTO IND. E COM. LTDA,BORRACHAS PLANALTO INDUSTRIA E COMERCIO EIRELI,**.*02.015/0001-**,6302015000190,76.0,1
4,4,COMERCIO DE METAIS MACIEL LTDA,VGM COMERCIO DE ALIMENTOS LTDA,**.*40.062/0001-**,31140062000122,77.0,1


Merge da base anterior com os dados de operações indiretas.

In [ ]:
ops = pd.merge(oper_automaticas, cnpj_receita[["cnpj_bndes", "cliente", "cnpj_receita", 'nome_receita']], how="left", on=["cnpj_bndes", "cliente"])

ops.head()

,cliente,cnpj_bndes,uf,municipio,municipio_codigo,data_da_contratacao,valor_da_operacao_em_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,forma_de_apoio,produto,instrumento_financeiro,inovacao,area_operacional,setor_cnae,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_do_agente_financeiro,situacao_da_operacao,cnpj_receita,nome_receita
0,BANCO COOPERATIVO SICOOB S.A.,**.*38.232/0001-**,SP,PEDREGULHO,3537008,2002-01-02,16000,"16000,0",RECURSOS LIVRES - TESOURO,TAXA FIXA,"8,75",24,36,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,OUTROS,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0119900,CULT PLANTAS LAVOURA TEMPORARIA NAO ESPECIFICA...,AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO COOPERATIVO SICOOB S.A.,02.038.232.0001-64,LIQUIDADA,2038232000164,BANCO COOPERATIVO DO BRASIL S/A
1,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,TRES DE MAIO,4321808,2002-01-02,8047,"8047,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,92816560000137,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL
2,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,8104,"8104,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,92816560000137,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL
3,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,ERECHIM,4307005,2002-01-02,6304,"6304,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",12,48,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,92816560000137,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL
4,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,**.*16.560/0001-**,RS,HUMAITA,4309704,2002-01-02,15000,"15000,0",RECURSOS VINCULADOS - FAT DEPÓSITOS ESPECIAIS,TAXA FIXA,"4,0",0,60,REEMBOLSÁVEL,INDIRETA,BNDES AUTOMÁTICO,PRONAF INVESTIMENTO,NÃO,AREA DE OPERACOES E CANAIS DIGITAIS,AGROPECUÁRIA E PESCA,Agropecuária,A0100000,"AGRICULTURA, PECUARIA E SERVICOS RELACIONADOS",AGROPECUÁRIA,AGROPECUÁRIA,MICRO,PRIVADA,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92.816.560.0001-37,LIQUIDADA,92816560000137,BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL


Obtendo dados de CNPJ's com a API Minha Receita: https://dev.to/camilacrdoso/obtendo-dados-de-cnpj-s-com-a-api-minha-receita-2hcd

In [ ]:
def get_cnpj_data(cnpj):
    minha_receita_api_url = 'https://minhareceita.org/'
    r = requests.post(minha_receita_api_url, data=cnpj, timeout=None)
    if r.status_code == 200:
        return json.loads(r.content)

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4, ensure_ascii=False)
    print(text)


Abaixo consta os dados de CNPJs que não são considerados poluidores, em contraponto à base de dados do IBAMA. 

In [ ]:
import pandas as pd 

outros_cnpjs = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/Cópia de bq-results-20210820-000530-nrr5rglojw3l.csv")
outros_cnpjs["potenc_poluic"] = "n"
dataset = outros_cnpjs[["cnpj", "potenc_poluic"]].append(poten_cnpj[["cnpj", "potenc_poluic"]].drop_duplicates())
dataset[["razao_social", "capital_social", "cep", "cnae_fiscal", "cnae_fiscal_descricao", "cnaes_secundarias", "codigo_municipio", "codigo_natureza_juridica", "porte", "municipio", "uf"]] = None
dataset = dataset.reset_index(drop=True)

Buscou-se mais CNPJs considerados poluidores pois se observou que os dados estavam desbalanceados, com viés para CNPJs não poluidores.

In [ ]:
ds_s = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/dataset_s.csv")
ds_s = ds_s[ds_s["razao_social"].notnull()].reset_index(drop=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(["potenc_poluic"], 1), dataset["potenc_poluic"], test_size=0.97, random_state=42)
x_train["target"] = y_train
dataset_tr = x_train

In [ ]:
for i in tqdm(dataset_s[dataset_s["razao_social"].isnull()].index):
  try:
    cnpj = {'cnpj': dataset_s["cnpj"].loc[i]}
    resp = get_cnpj_data(cnpj)

    dataset_s["razao_social"].loc[i] = resp["razao_social"]
    dataset_s["capital_social"].loc[i] = resp["capital_social"]
    dataset_s["cep"].loc[i] = resp["cep"]
    dataset_s["cnae_fiscal"].loc[i] = resp["cnae_fiscal"]
    dataset_s["cnae_fiscal_descricao"].loc[i] = resp["cnae_fiscal_descricao"]
    dataset_s["codigo_municipio"].loc[i] = resp["codigo_municipio"]
    dataset_s["codigo_natureza_juridica"].loc[i] = resp["codigo_natureza_juridica"]
    dataset_s["cnaes_secundarias"].loc[i] = resp["cnaes_secundarias"]
    dataset_s["porte"].loc[i] = resp["porte"]
    dataset_s["municipio"].loc[i] = resp["municipio"]
    dataset_s["uf"].loc[i] = resp["uf"]
  
  except:
    pass

100%|██████████| 19164/19164 [1:51:38<00:00,  2.86it/s]


In [ ]:
dataset_s.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/dataset_s.csv", index=False)

In [ ]:
for i in tqdm(dataset_tr[dataset_tr["razao_social"].isnull()].index):
  try:
    cnpj = {'cnpj': dataset_tr["cnpj"].loc[i]}
    resp = get_cnpj_data(cnpj)

    dataset_tr["razao_social"].loc[i] = resp["razao_social"]
    dataset_tr["capital_social"].loc[i] = resp["capital_social"]
    dataset_tr["cep"].loc[i] = resp["cep"]
    dataset_tr["cnae_fiscal"].loc[i] = resp["cnae_fiscal"]
    dataset_tr["cnae_fiscal_descricao"].loc[i] = resp["cnae_fiscal_descricao"]
    dataset_tr["codigo_municipio"].loc[i] = resp["codigo_municipio"]
    dataset_tr["codigo_natureza_juridica"].loc[i] = resp["codigo_natureza_juridica"]
    dataset_tr["cnaes_secundarias"].loc[i] = resp["cnaes_secundarias"]
    dataset_tr["porte"].loc[i] = resp["porte"]
    dataset_tr["municipio"].loc[i] = resp["municipio"]
    dataset_tr["uf"].loc[i] = resp["uf"]
  
  except:
    pass


  0%|          | 0/82260 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
 49%|████▉     | 40554/82260 [4:27:02<3:15:30,  3.56it/s]

Finalmente, o dataset de treinamento salvo.

In [ ]:
dataset_tr.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/dataset_tr.csv")

### Import dos dados e divisão de dados de treino e teste

Os dados a seguir importam tanto dados de treino contidos no dataset_tr.csv quanto em dataset_s.csv. Este último foi agregado para tornar o target do modelo mais balanceado, entre empresas poluidoras e não poluidoras. 

Posteriormente, faz-se o append desses dois datasets e o trata incluindo 0 em valores faltantes. A escolha deste tratamento foi arbitrária. 

Por fim, fez o drop de três colunas - porte, municipio e uf - devido a problemas na coleta desses dados na API Minha Receita.

Ambos datasets tiveram as colunas com razao social dropadas devido ao fato de serem esses cnpjs que a API do Minha Receita não conseguiu dados.

In [ ]:
## import dados de treino sem target balanceado

train = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/dataset_tr.csv")
train = train[train["razao_social"].notnull()]

## import dados com variáveis de target "s" para balancear dataset anterior

ds_s = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/dataset_s.csv")
ds_s = ds_s[ds_s["razao_social"].notnull()].reset_index(drop=True)

## append dos dois datasets e o drop das colunas

train = train.append(ds_s).reset_index(drop=True)
train[["cep", "cnae_fiscal", "codigo_municipio", "codigo_natureza_juridica", "porte"]] = train[["cep", "cnae_fiscal", "codigo_municipio", "codigo_natureza_juridica", "porte"]].fillna(0).astype(int)
train = train.drop(["porte", "municipio", "uf"], 1)

Abaixo consta o código de amostragem da base em treino e teste. O volume de treino em 0.2 foi um valor arbitrário, mas o objetivo era concentrar ao máximo dados em treino para o algoritmo de AutoML modelar a bem a curva, e como os dados possuem um volume significativo, acreditamos que ter 20% da base em teste não atrapalharia a avaliação de métricas de desempenho.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train.drop(["target"], 1), train["target"], test_size=0.20, random_state=42)
x_train["target"] = y_train
x_test["target"] = y_test

## Modelagem 

Para se desenvolver este modelo, optou-se por uma biblioteca de AutoML chamada [LightAutoML](https://github.com/sberbank-ai-lab/LightAutoML). Com ela é posssível conciliar em dados de tabulares de maneira simples dados não estruturados, como textos, com dados estruturados, como o código do municipio. 

Para se fazer essa conciliação, utilizou-se da abordagem presente no exemplo deste [artigo](https://github.com/sberbank-ai-lab/LightAutoML/blob/master/Tutorial_5.%20NLP%20Interpretation.ipynb). Inclusive se utilizou os mesmos parametros para o modelo.

Para processar os textos, utilizou-se dos pesos do modelo pré-treinado ['bert-base-multilingual-cased'](https://huggingface.co/bert-base-multilingual-cased), desenvolvido pelo Google Research. Com esse modelo, é posssível vetorizar os textos da Razão Social, CNAE Fiscal e secundária, tornando as variáveis mais assertivas para o algoritmo. 



In [ ]:
## definição da função de algumas colunas do dataset 
### dropou-se o numero do cnpj devido a pouco valor preditivo e ainda poderia ter viés

roles = {
    'text': ["razao_social", "cnae_fiscal_descricao", "cnaes_secundarias"], 
    'drop': ['cnpj'],
    'target': 'target'
}

## definição do problema: classificação binária (empresa é ou não potencialmente poluidora?)

task = Task('binary')

## definição dos par

automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='0',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']]
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'multi',
        'bert_model': 'bert-base-multilingual-cased'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 35,
    },
    verbose=2
)

oof_pred = automl.fit_predict(x_train, roles=roles)

Model language mode: multi
Start automl preset with listed constraints:
- time: 3600 seconds
- cpus: 1 cores
- memory: 16 gb

Train data shape: (60054, 10)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 3598.8453130722046 secs
Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN =====



val: 100%|██████████| 626/626 [00:03<00:00, 191.83it/s]


Epoch: 0, train loss: 0.647153913974762, val loss: 0.6251937747001648, val metric: 0.9133855124611803


val: 100%|██████████| 626/626 [00:03<00:00, 198.93it/s]


Epoch: 1, train loss: 0.6054608225822449, val loss: 0.586753785610199, val metric: 0.9155405257056539


...


val: 100%|██████████| 626/626 [00:03<00:00, 202.12it/s]

Epoch: 33, train loss: 0.3063644468784332, val loss: 0.3044414520263672, val metric: 0.9413176328695978



val: 100%|██████████| 626/626 [00:03<00:00, 202.66it/s]

Epoch: 34, train loss: 0.3055269718170166, val loss: 0.3035410940647125, val metric: 0.9414398680245055


Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
Time left 1376.1329708099365

Automl preset training completed in 2223.90 seconds.


### Evaluate do modelo

Para avaliar se o modelo de classificação performa bem, principalmente para problemas que envolvam classificação binária, utiliza-se, comumente, a curva ROC e o AUC score. Basicamente é uma métrica que avalia o quão poderoso é o modelo para lidar com problema de escolhas binárias, e varia entre 0 e 1. Neste modelo, observa-se que há uma performance bastante satisfatória, com o AUC de teste de ˜0.95.

In [ ]:
test_pred = automl.predict(x_test)

print('Check score:')
print('TRAIN score: {}'.format(roc_auc_score(x_train[roles['target']].values, oof_pred.data[:, 0])))
print('TEST score: {}'.format(roc_auc_score(x_test[roles['target']].values, test_pred.data[:, 0])))

test: 100%|██████████| 470/470 [00:02<00:00, 180.97it/s]


Check score:
TRAIN score: 0.9412481841146614
TEST score: 0.9532557801517938


Outras métricas tambéém são importantes, como Precisão, Recall e o F1-Score, e todas atestam a qualidade o modelo.

In [ ]:
preds = pd.DataFrame(columns = ["y", "predict_proba"])
preds["y"] = x_test[roles['target']].values
preds["predict_proba"] = test_pred.data
preds["y_predict"] = ["s" if x > 0.5 else "n" for x in preds["predict_proba"]]

print(classification_report(preds["y"], preds["y_predict"]))

              precision    recall  f1-score   support

           n       0.91      0.92      0.91      9247
           s       0.87      0.85      0.86      5767

    accuracy                           0.89     15014
   macro avg       0.89      0.89      0.89     15014
weighted avg       0.89      0.89      0.89     15014



In [ ]:
ops_cnpjs = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/cnpjs_ops1.csv")
ops_cnpjs = ops_cnpjs[ops_cnpjs["razao_social"].notnull()]
ops_cnpjs[["cep", "cnae_fiscal", "codigo_municipio", "codigo_natureza_juridica"]] = ops_cnpjs[["cep", "cnae_fiscal", "codigo_municipio", "codigo_natureza_juridica"]].fillna(0).astype(int)
ops_cnpjs = ops_cnpjs.drop(["municipio", "uf"], 1)

In [ ]:
preds_ops = automl.predict(ops_cnpjs)

test: 100%|██████████| 2272/2272 [00:10<00:00, 220.91it/s]


In [ ]:
ops_cnpjs["cnpj_polui_prob"] = preds_ops.data[:, 0]
ops_cnpjs["cnpj_polui"] = ["s" if x > 0.5 else "n" for x in ops_cnpjs["cnpj_polui_prob"]]
ops_cnpjs.head()

Pela distribuição da identificação de empresas poluidoras ou não, é possível observar que ficou bem distribuído.

In [ ]:
ops_cnpjs["cnpj_polui"].value_counts()

n    38799
s    33882
Name: cnpj_polui, dtype: int64

### Merge com dados de operações e geolocalização dos municipios

Por fim, criou-se o dataset principal com merge com dados de operações indiretas e também com geolocalização (latitude e longitude) dos municípios reconhecidos na operação.

A escolha dos dados a partir de 2016 se deu por ser um dataset muito grande

In [ ]:
ops_cnpjs = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/ops_cnpjs_polui.csv")
ops['data_da_contratacao'] = pd.to_datetime(ops['data_da_contratacao'])
ops_cnpjs["cnpj"] = ops_cnpjs["cnpj"].astype(str)
ops_cnpj_merge = pd.merge(ops[ops['data_da_contratacao']>"2016-01-01"].rename(columns = {"cnpj_receita": "cnpj"}), ops_cnpjs, how = "left", on = ["cnpj"])
ops_cnpj_merge = ops_cnpj_merge[ops_cnpj_merge["razao_social"].notnull()]

municipios = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_municipios/municipios.csv")
municipios = municipios[["codigo_ibge", "latitude", "longitude"]]
municipios = municipios.rename(columns = {"codigo_ibge": "municipio_codigo"})

ops_cnpj_merge = pd.merge(ops_cnpj_merge, municipios, how = "left", on = ["municipio_codigo"])

ops_cnpj_merge.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj_potenciais/ops_cnpj_completo.csv", index=False)